In [ ]:
import numpy as np
from PyQt5 import uic
import sys
import pylab as plt
import time
from PyQt5.QtWidgets import QMainWindow, QLabel, QCheckBox, QWidget
from PyQt5.QtCore import Qt, QSize
from PyQt5 import QtCore, QtWidgets, QtGui
from matplotlib.backends.backend_qtagg import FigureCanvasQTAgg
from matplotlib.backends.backend_template import FigureCanvas
from matplotlib.figure import Figure
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tqdm.notebook import tqdm
import segyio
import pandas as pd
import os

from ui_file_demidol import Ui_MainWindow
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QFileDialog, QVBoxLayout, QTabWidget
from matplotlib.backends.backend_qt5 import NavigationToolbar2QT as NavigationToolbar


class MplCanvas(FigureCanvasQTAgg):
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height))
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)


#     def __init__(self, parent=None):
#         fig = Figure(figsize=(5, 5))
#         self.axes = fig.add_subplot(111)
#         super(MplCanvas, self).__init__(fig)


class SlowTask(QtCore.QThread):
    updated = QtCore.pyqtSignal(int)
    running = False

    def __init__(self, *args, **kwargs):
        super(SlowTask, self).__init__(*args, **kwargs)
        self.percent = 0
        self.running = True

    def run(self):
        while self.running:
            if self.percent == 99:
                self.stop()
            self.percent += 1
            self.percent %= 100
            self.updated.emit(int(self.percent))
            time.sleep(0.01)

    def stop(self):
        self.running = False


class MyWidget(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.load_file_button.clicked.connect(self.load_file)
        self.horizontalSlider.valueChanged.connect(self.plot)
        self.verticalSlider.valueChanged.connect(self.plot)
        self.progressBar.setProperty("value", 0)
        self.sc = MplCanvas(self)
        self.comb_box_value = 'SNR'

        # self.horizontalSlider.valueChanged.connect(self.sliderchange)

        self.T0_n = 0
        self.T1_n = 0
        self.N0_n = 0
        self.N1_n = 0
        self.T0_s = 0
        self.T1_s = 0
        self.N0_s = 0
        self.N1_s = 0
        self.change()

    def on_update(self, data):
        self.progressBar.setValue(data)

    #     def load_file(self):
    #         self.listWidget.clear()
    #         fname, filetype = QFileDialog.getOpenFileNames(
    #             self,
    #             "Open file"
    #         )
    #         self.mass_with_filesnames = []
    #         self.mass_with_loaded = []
    #         self.mass_with_loaded2 = []
    #         for i in fname:
    #             self.mass_with_filesnames.append(i.split("/")[-1])
    #             self.mass_with_loaded.append(np.load(i))
    #             self.mass_with_loaded2.append(np.load(i))

    #         self.listWidget.itemClicked.connect(self.clicked_to_image)
    #         self.listWidget.itemClicked.connect(self.change)
    #         self.comboBox.currentTextChanged.connect(self.comb_box)
    #         self.listWidget.addItems(self.mass_with_filesnames)
    #         self.task = SlowTask(self)
    #         self.task.updated.connect(self.on_update)

    #         self.task.start()

    def load_file(self, *args, **kwargs):
        self.fname, filetype = QFileDialog.getOpenFileNames(
            self,
            "Open file"
        )
        self.files = []
        self.mass_with_loaded = []
        for i in range(len(self.fname)):
            name = self.fname[i].split("/")[-1]
            print(self.fname[i])
            # self.files.append(np.load(self.fname[i]))
            self.listWidget.addItem(name)
            filename = os.path.normpath(name)
            print('okey')
            segy = segyio.open(filename, ignore_geometry=True)
            print('ok')
            TRACE_NUMBER = segy.attributes(segyio.TraceField.TraceNumber)[:]  # Номер трассы
            self.TRACE_NUMBER_MIN = min(TRACE_NUMBER)
            self.TRACE_NUMBER_MAX = max(TRACE_NUMBER)
            FFID = segy.attributes(segyio.TraceField.FieldRecord)[:]  # Номер источника (номер сейсмограммы)
            SX = segy.attributes(segyio.TraceField.SourceX)[:]  # Координата источника по Х
            SY = segy.attributes(segyio.TraceField.SourceY)[:]  # Координата источника по Y
            ELEVATION = segy.attributes(segyio.TraceField.ReceiverDatumElevation)[:]  # Высота источника
            head = pd.DataFrame(data=np.array([TRACE_NUMBER,
                                               FFID,
                                               SX,
                                               SY,
                                               ELEVATION]).T,
                                columns=['Trace_number',
                                         'FFID',
                                         'SouX',
                                         'SouY',
                                         'Elev'])
            FFID_unique = head.FFID.unique()
            self.seism = []
            for i in tqdm(range(len(FFID_unique))):
                index_seism = head[head['FFID'] == FFID_unique[i]].index.values
                self.seism.append(np.array([segy.trace[ind] for ind in index_seism]))
            self.mass_with_loaded.append(self.seism)
            self.dt = segy.samples[1] / 1000  # Шаг дескритизации по времени
            self.N = len(segy.trace[0])  # Длина трассы

        self.horizontalSlider.setMinimum(1)
        self.horizontalSlider.setMaximum(len(self.seism))
        self.verticalSlider.setMinimum(1)
        self.verticalSlider.setMaximum(1000)
        print('load_file')
        self.toggle_buttons()
        self.task = SlowTask(self)
        self.task.updated.connect(self.on_update)
        self.listWidget.itemClicked.connect(self.clicked_to_image)
        self.listWidget.itemClicked.connect(self.change)
        self.comboBox.currentTextChanged.connect(self.comb_box)
        # self.listWidget.addItems(self.mass_with_filesnames)
        self.task = SlowTask(self)
        self.task.start()

    def toggle_buttons(self):
        self.load_file_button.setEnabled(not self.load_file_button.isEnabled())

    def plot(self, _translate):
        # очистка виджета
        for i in reversed(range(self.verticalLayout.count())):
            self.verticalLayout.itemAt(i).widget().deleteLater()
        self.index = self.horizontalSlider.value() - 1
        self.indexver = self.verticalSlider.value()
        print(self.index)
        sc = MplCanvas(self, width=5, height=4, dpi=100)
        vmin = np.min(self.seism[self.index]) / self.indexver
        vmax = np.max(self.seism[self.index]) / self.indexver
        sc.axes.imshow(self.seism[self.index].T,
                       aspect='auto',
                       cmap='seismic',
                       vmin=vmin,
                       vmax=vmax,
                       extent=[self.TRACE_NUMBER_MIN, self.TRACE_NUMBER_MAX, self.N * self.dt, 0])
        title = 'CDP gather №' + str(self.index)
        sc.axes.set_title(title)
        sc.axes.set_xlabel('Trace number')
        sc.axes.set_ylabel('Time, s')
        sc.axes.set_position([0.1, 0.2, 0.85, 0.75])
        self.verticalLayout.addWidget(sc)
        self.show()

        #     def load_file(self):

    #         self.listWidget.clear()
    #         fname, filetype = QFileDialog.getOpenFileNames(
    #             self,
    #             "Open file"
    #         )
    #         self.mass_with_filesnames = []
    #         self.mass_with_loaded = []
    #         self.mass_with_loaded2 = []
    #         for nu in fname:
    #             self.mass_with_filesnames.append(nu.split("/")[-1])
    #             segy = segyio.open(nu, ignore_geometry=True)
    #             consts = np.zeros(len(segy.trace), dtype=float)
    #             TRACE_NUMBER = segy.attributes(segyio.TraceField.TraceNumber)[:]
    #             FFID = segy.attributes(segyio.TraceField.FieldRecord)[:]
    #             SX = segy.attributes(segyio.TraceField.SourceX)[:]
    #             SY = segy.attributes(segyio.TraceField.SourceY)[:]
    #             ELEVATION = segy.attributes(segyio.TraceField.ReceiverDatumElevation)[:]
    #             head = pd.DataFrame(data=np.array([TRACE_NUMBER,
    #                                                FFID,
    #                                                SX,
    #                                                SY,
    #                                                ELEVATION]).T,
    #                                 columns=["Trace_nubber",
    #                                          "FFID",
    #                                          "SouX", "SouY", "Elev"])
    #             FFID_unique = head.FFID.unique()
    #             self.seism = []
    #             for i in range((len(FFID_unique))):
    #                 index_seism = head[head["FFID"] == FFID_unique[i]].index.values
    #                 self.seism.append(np.array([segy.trace[j] for j in index_seism]))
    #             self.mass_with_loaded.append(self.seism)
    #             # for j in segy.trace:
    #             #     self.mass_with_loaded.append(j)
    #             #     self.mass_with_loaded2.append(j)
    #         self.listWidget.itemClicked.connect(self.sliderchange)
    #         self.listWidget.itemClicked.connect(self.change)
    #         self.listWidget.addItems(self.mass_with_filesnames)
    #         self.comboBox.currentTextChanged.connect(self.comb_box)
    #         self.task = SlowTask(self)
    #         self.task.updated.connect(self.on_update)

    #         self.task.start()

    #     def sliderchange(self):
    #         self.horizontalSlider.setRange(0, len(self.mass_with_loaded[self.listWidget.currentRow()]))
    #         for i in reversed(range(self.verticalLayout.count())):
    #             self.verticalLayout.itemAt(i).widget().deleteLater()
    #         self.tabwidget = QTabWidget()
    #         sc = MplCanvas(self)
    #         lim = 1e3
    #         vmin = np.min(self.mass_with_loaded[self.listWidget.currentRow()][self.horizontalSlider.value() - 1] / lim)
    #         vmax = np.max(self.mass_with_loaded[self.listWidget.currentRow()][self.horizontalSlider.value() - 1] / lim)
    #         sc.axes.imshow(self.mass_with_loaded[self.listWidget.currentRow()][self.horizontalSlider.value() - 1].T,
    #                        aspect="auto",
    #                        cmap="seismic",
    #                        vmin=vmin,
    #                        vmax=vmax)
    #         sc.axes.set_xlabel('Trace number')
    #         sc.axes.set_ylabel('Time, ms')
    #         # self.tabwidget.addTab(sc, "before")
    #         # self.tabwidget.addTab(sc, "after")
    #         # self.verticalLayout.addWidget(self.tabwidget)

    def change(self):
        self.T0_noise.valueChanged.connect(self.clicked_to_image)
        self.T1_noise.valueChanged.connect(self.clicked_to_image)
        self.N0_noise.valueChanged.connect(self.clicked_to_image)
        self.N1_noise.valueChanged.connect(self.clicked_to_image)
        self.T0_signal.valueChanged.connect(self.clicked_to_image)
        self.T1_signal.valueChanged.connect(self.clicked_to_image)
        self.N0_signal.valueChanged.connect(self.clicked_to_image)
        self.N1_signal.valueChanged.connect(self.clicked_to_image)

    def comb_box(self):
        if self.comboBox.currentText() == 'SNR':
            self.comb_box_value = 'SNR'
        elif self.comboBox.currentText() == 'None':
            self.comb_box_value = 'none'
        self.clicked_to_image()

    def work(self):
        if self.comb_box_value == 'SNR':
            self.use_snr()
        elif self.comb_box_value == 'none':
            self.non()

        for i in reversed(range(self.verticalLayout.count())):
            self.verticalLayout.itemAt(i).widget().deleteLater()

        self.toolbar = NavigationToolbar(self.sc, self)
        self.verticalLayout.addWidget(self.toolbar)
        self.verticalLayout.addWidget(self.sc)

        self.show()

    def clicked_to_image(self):
        self.tabwidget = QTabWidget()
        for i in reversed(range(self.verticalLayout.count())):
            self.verticalLayout.itemAt(i).widget().deleteLater()

        self.sc = MplCanvas(self)
        print(type(np.array(self.mass_with_loaded[self.listWidget.currentRow()])))
        self.sc.axes.imshow(np.array(self.mass_with_loaded[self.listWidget.currentRow()]), aspect="auto",
                            cmap="seismic")
        self.sc.axes.set_xlabel('Trace number')
        self.sc.axes.set_ylabel('Time, ms')

        self.data_clear = np.zeros_like(self.mass_with_loaded[self.listWidget.currentRow()])

        self.T0_noise.setRange(0, self.data_clear.shape[0])
        self.T1_noise.setRange(0, self.data_clear.shape[0])
        self.N0_noise.setRange(0, self.data_clear.shape[1])
        self.N1_noise.setRange(0, self.data_clear.shape[1])

        self.T0_signal.setRange(0, self.data_clear.shape[0])
        self.T1_signal.setRange(0, self.data_clear
                                .shape[0])
        self.N0_signal.setRange(0, self.data_clear.shape[1])
        self.N1_signal.setRange(0, self.data_clear.shape[1])

        if self.T0_n != self.T0_noise.value() or \
                self.T1_n != self.T1_noise.value() or \
                self.N0_n != self.N0_noise.value() or \
                self.N1_n != self.N1_noise.value() or \
                self.T0_s != self.T0_signal.value() or \
                self.T1_s != self.T1_signal.value() or \
                self.N0_s != self.N0_signal.value() or \
                self.N1_s != self.N1_signal.value():
            self.T0_n = self.T0_noise.value()
            self.T1_n = self.T1_noise.value()
            self.N0_n = self.N0_noise.value()
            self.N1_n = self.N1_noise.value()

            self.T0_s = self.T0_signal.value()
            self.T1_s = self.T1_signal.value()
            self.N0_s = self.N0_signal.value()
            self.N1_s = self.N1_signal.value()
        # self.change()

        if self.comb_box_value == 'SNR':
            self.use_snr()
        elif self.comb_box_value == 'none':
            self.non()

        for i in reversed(range(self.verticalLayout.count())):
            self.verticalLayout.itemAt(i).widget().deleteLater()

        self.toolbar = NavigationToolbar(self.sc, self)
        self.verticalLayout.addWidget(self.toolbar)
        self.verticalLayout.addWidget(self.sc)

        self.show()

    def draw(self):
        self.sc.axes.add_patch(
            patches.Rectangle((self.N0_n, self.T0_n), self.N1_n - self.N0_n, self.T1_n - self.T0_n, linewidth=1,
                              edgecolor='black', facecolor='none'))

        self.sc.axes.add_patch(
            patches.Rectangle((self.N0_s, self.T0_s), self.N1_s - self.N0_s, self.T1_s - self.T0_s, linewidth=1,
                              edgecolor='black', facecolor='none'))
        self.sc_snr = MplCanvas(self)
        self.sc_snr.axes.imshow(self.data_clear, aspect="auto", cmap="seismic")
        self.sc_snr.axes.set_xlabel('Trace number')
        self.sc_snr.axes.set_ylabel('Time, ms')

    def non(self):
        non_value = 'none'
        self.label_3.setText(f"None = {non_value}")

        self.draw()

    def snr(self, T0_s, T1_s, N0_s, N1_s, T0_n, T1_n, N0_n, N1_n):
        data = self.mass_with_loaded[self.listWidget.currentRow()]
        if sum([T0_s, T1_s, N0_s, N1_s, T0_n, T1_n, N0_n, N1_n]) == 0:
            return 0

        energy_n = np.sqrt(np.sum(list(map(lambda i: i ** 2, data[T0_n:T1_n, N0_n:N1_n]))))
        energy_s = np.sqrt(np.sum(list(map(lambda i: i ** 2, data[T0_s:T1_s, N0_s:N1_s]))))

        #         self.sc.axes.add_patch(patches.Rectangle((N0_n, T0_n), N1_n-N0_n, T1_n-T0_n, linewidth=1, edgecolor='black', facecolor='none'))

        #         self.sc.axes.add_patch(patches.Rectangle((N0_s, T0_s), N1_s-N0_s, T1_s-T0_s, linewidth=1, edgecolor='black', facecolor='none'))

        return energy_s / energy_n

    def use_snr(self):
        self.data_clear = np.zeros_like(self.mass_with_loaded[self.listWidget.currentRow()])

        snr_value = self.snr(self.T0_s, self.T1_s, self.N0_s, self.N1_s, self.T0_n, self.T1_n, self.N0_n, self.N1_n)

        self.label_3.setText(f"SNR = {snr_value}")

        # Визуализация
        self.draw()


#         self.sc_snr = MplCanvas(self)
#         self.sc_snr.axes.imshow(self.data_clear, aspect="auto", cmap="seismic")
#         self.sc_snr.axes.set_xlabel('Trace number')
#         self.sc_snr.axes.set_ylabel('Time, ms')


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyWidget()
    ex.setMinimumSize(950, 650)
    ex.show()
    sys.exit(app.exec_())
